Checking general difficulty of 4 different CO classes.

In [ ]:
%load_ext autoreload
%autoreload
from retro_branching.environments import EcoleBranching, EcoleConfiguring
from retro_branching.agents import StrongBranchingAgent, PseudocostBranchingAgent, RandomAgent
from retro_branching.utils import seed_stochastic_modules_globally
from retro_branching.rewards import NormalisedLPGain

import ecole
import numpy as np
import random
import copy
import time

import networkx as nx
from networkx.algorithms.shortest_paths.generic import shortest_path
from networkx.algorithms.traversal.depth_first_search import dfs_tree
from networkx.drawing.nx_pydot import graphviz_layout
import matplotlib.pyplot as plt

seed = 0
seed_stochastic_modules_globally(default_seed=seed)

In [ ]:
%autoreload
agent = StrongBranchingAgent()

env = EcoleBranching(observation_function='default',
                      information_function='default',
                      reward_function='default',
                      scip_params='gasse_2019')
env.seed(seed)

co_classes = {'set_covering': ecole.instance.SetCoverGenerator(n_rows=500, n_cols=1000, density=0.05),
                'combinatorial_auction': ecole.instance.CombinatorialAuctionGenerator(n_items=100, n_bids=500),
                'capacitated_facility_location': ecole.instance.CapacitatedFacilityLocationGenerator(n_customers=100, n_facilities=100),
                'maximum_independent_set': ecole.instance.IndependentSetGenerator(n_nodes=100)}

num_tests = 5

In [ ]:
co_class_stats = {key: {'num_nodes': [], 'solving_time': []} for key in co_classes.keys()}

for co_class in co_classes.keys():
    print(f'\n >> Solving {num_tests} {co_class} instances... <<')
    num_episodes = 0
    done = True
    while num_episodes < num_tests:
        # reset env
        print(f'Resetting {co_class} instance...')
        while done:
            env.seed(seed)
            instance = next(co_classes[co_class])
            agent.before_reset(instance)
            obs, action_set, reward, done, info = env.reset(instance)
            
        # solve instance
        print(f'{co_class} instance reset. Solving instance...')
        start_t = time.time()
        while not done:
            action, action_idx = agent.action_select(action_set, model=env.model, done=done)
            obs, action_set, reward, done, info = env.step(action)

        # record stats
        solving_time = time.time() - start_t
        num_nodes = info['num_nodes']
        num_episodes += 1
        print(f'{co_class} instance {num_episodes} of {num_tests} solved in {solving_time:.3f} s with {num_nodes} nodes')
        co_class_stats[co_class]['num_nodes'].append(num_nodes)
        co_class_stats[co_class]['solving_time'].append(solving_time)

In [ ]:
for co_class in co_class_stats.keys():
    print(f'{co_class} | mean num_nodes: {np.mean(co_class_stats[co_class]["num_nodes"])} | mean solving_time: {np.mean(co_class_stats[co_class]["solving_time"]):.3f} s')